<font color="#4b76b7">To start practicing, you will need to make a copy of it. Go to File > Save a Copy in Drive. You can then use the new copy that will appear in the new tab.</font>


# AfterWork Data Science: Getting Started with NLP Project

### Prerequisites

In [1]:
# Importing the required libraries
# ---
# 
import pandas as pd # library for data manipulation
import numpy as np  # librariy for scientific computations
import re           # regex library to perform text preprocessing
import string       # library to work with strings
import nltk         # library for natural language processing
import scipy        # scientific conputing 

### 1. Importing our Data

In [3]:
# Question: Given a new tweets, create a sentiment analysis model that will 
# predict whether a tweet will contain positive or negative sentiment.
# ---
# Dataset url = https://bit.ly/31kqByD 
# ---
#
df = pd.read_csv('https://bit.ly/31kqByD', encoding='latin-1',header=None)
df.head()

,0,1,2,3,4,5,6
0,NaN,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,346508.0,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
2,883537.0,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
3,764173.0,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
4,638701.0,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...


### 2. Data Exploration

In [4]:
# We can determine the size of our dataset
# ---
#
df.shape

(10001, 7)

Seems this dataset will need some data cleaning i.e. columns. We also don't need some columns to perform create our model. We will drop those columns.

### 3. Data Preparation

#### Basic Data Cleaning Techniques

In [5]:
# We rename the columns for ease of referencing our columns later on
# ---
#
df.columns = ['id', 'target', 't_id', 'created_at', 'query', 'user', 'text']
df.head()

,id,target,t_id,created_at,query,user,text
0,NaN,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,346508.0,0,2016177685,Wed Jun 03 06:18:50 PDT 2009,NO_QUERY,UriGrey,Obama forges his Muslim alliance against the c...
2,883537.0,4,1686152287,Sun May 03 04:02:08 PDT 2009,NO_QUERY,MariesolW,Had the most spectacular prom ever but now my...
3,764173.0,0,2298725623,Tue Jun 23 12:02:12 PDT 2009,NO_QUERY,ColleenBurns,I am overwhelmed today taking a moment to eat...
4,638701.0,0,2234530495,Thu Jun 18 23:13:54 PDT 2009,NO_QUERY,queenarchy,@lindork Tres sad. I was totally a Max fan. #...


In [6]:
# We retain the relevant columns by dropping the columns we don't need 
# for creating a sentiment analysis model. 
# ---
#
df = df.drop(['id', 't_id', 'created_at', 'query', 'user'], axis = 1)
df.head()

,target,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,Obama forges his Muslim alliance against the c...
2,4,Had the most spectacular prom ever but now my...
3,0,I am overwhelmed today taking a moment to eat...
4,0,@lindork Tres sad. I was totally a Max fan. #...


In [7]:
# Understanding the distribution of target
# ---
#
df.target.value_counts() 

0    5068
4    4933
Name: target, dtype: int64

In [8]:
# Let's determine whether our columns have the right data types
# ---
#
df.dtypes

target     int64
text      object
dtype: object

In [9]:
# What values are in our target variable?
# ---
#
df.target.unique()

array([0, 4])

These are the two classes to which each document (text) belongs. The target value 0 means a text with a negative sentiment, while that of 4 means a text with a positive sentiment. 

In [10]:
# Let's check for missing values 
# ---
# 
df.isnull().sum()

target    0
text      0
dtype: int64

We don't have any missing values, so we are good to go.

#### Text Processing

In [11]:
# Text Cleaning: Removing all urls/links
# ---
# 
df['text'] =  df['text'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+','', str(x)))
df[['text']].head()

,text
0,@switchfoot - A that's a bummer. You shoulda...
1,Obama forges his Muslim alliance against the c...
2,Had the most spectacular prom ever but now my...
3,I am overwhelmed today taking a moment to eat...
4,@lindork Tres sad. I was totally a Max fan. #...


In [21]:
# Text Cleaning: Removing @ and # characters or replace them with space
# ---
# YOUR CODE GOES BELOW
df['text'] =  df['text'].str.replace('@',' ').str.replace('#',' ')
df.head(5)


,target,text
0,0,switchfoot - a that's a bummer. you shoulda...
1,0,obama forges his muslim alliance against the c...
2,4,had the most spectacular prom ever but now my...
3,0,i am overwhelmed today taking a moment to eat...
4,0,lindork tres sad. i was totally a max fan. ...


In [22]:
# Text Cleaning: Conversion to lowercase
# ---
# YOUR CODE GOES BELOW
df['text'] =  df['text'].str.lower()


In [32]:
pip install wordninja

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 541 kB 5.1 MB/s 
  Created wheel for wordninja: filename=wordninja-2.0.0-py3-none-any.whl size=541551 sha256=63bd9d766828d0e9561a9fb38cfdf53671d8d23277a18d8f42122b0526cb0c1a
  Stored in directory: /root/.cache/pip/wheels/dd/3f/eb/a2692e3d2b9deb1487b09ba4967dd6920bd5032bfd9ff7acfc
Successfully built wordninja


In [71]:
from textblob import TextBlob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [36]:
# Performing the split
# ---
#
df['split_text']=df['text'].str.split()
df.head()

,target,text,split_text
0,0,switchfoot - a that's a bummer. you shoulda...,"[switchfoot, -, a, that's, a, bummer., you, sh..."
1,0,obama forges his muslim alliance against the c...,"[obama, forges, his, muslim, alliance, against..."
2,4,had the most spectacular prom ever but now my...,"[had, the, most, spectacular, prom, ever, but,..."
3,0,i am overwhelmed today taking a moment to eat...,"[i, am, overwhelmed, today, taking, a, moment,..."
4,0,lindork tres sad. i was totally a max fan. ...,"[lindork, tres, sad., i, was, totally, a, max,..."


In [45]:
df.drop(columns=['split_text'],inplace=True)

In [40]:
# Text Cleaning: Removing punctuation characters
# ---
# YOUR CODE GOES BELOW
#
df['text'] = df['text'].str.replace('[^\w\s]','')
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


,target,text,split_text
0,0,switchfoot a thats a bummer you shoulda go...,"[switchfoot, -, a, that's, a, bummer., you, sh..."
1,0,obama forges his muslim alliance against the c...,"[obama, forges, his, muslim, alliance, against..."
2,4,had the most spectacular prom ever but now my...,"[had, the, most, spectacular, prom, ever, but,..."
3,0,i am overwhelmed today taking a moment to eat...,"[i, am, overwhelmed, today, taking, a, moment,..."
4,0,lindork tres sad i was totally a max fan sy...,"[lindork, tres, sad., i, was, totally, a, max,..."


In [57]:
# Text Cleaning: Removing stop words
# ---
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
stop = stopwords.words('english')

df['no_of_stopwords'] = df.text.apply(lambda x: len([x for x in x.split() if x in stop]))
df[['text','no_of_stopwords']].sample(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,no_of_stopwords
5561,whitneypannell sounds adventuroushave fun,0
3698,im online,0
5233,bumper to bumper traffic on the fdr watching e...,6
9855,kevinqnguyen windows 7 is pretty cool i insta...,5
5983,sad face back to work tomor4days and then bre...,3
8867,woo followers,0
4064,smokin in a bar again yippie,3
2593,lukasandnina glad you enjoyed thanks again,2
4633,baby gauges are so cute,2
7263,chrisportela that sucks im sorry,1


In [59]:
# Text Cleaning: Lemmatization
# ---
# YOUR CODE GOES BELOW
#
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
# For lemmatization, we will need to download wordnet

from textblob import Word

# Lemmatizing our text
df['lemmatization'] = df.text.apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 
df[['text', 'lemmatization']].sample(10)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,text,lemmatization
90,starving at the gym wishing he had breakfast,starving at the gym wishing he had breakfast
6957,rob3taylor awww thanks,rob3taylor awww thanks
5186,deefamouss thankiezeee hiunny follow me pumpk...,deefamouss thankiezeee hiunny follow me pumpki...
1318,beccainbalt im the one in pinkblack on the le...,beccainbalt im the one in pinkblack on the lef...
7523,is dumb tite i cant fine my headscarf i cant g...,is dumb tite i cant fine my headscarf i cant g...
5956,jlsrockmycowsox yeah i saw that tooo aston as...,jlsrockmycowsox yeah i saw that tooo aston a m...
6196,tabbybottoms hi i love you,tabbybottoms hi i love you
5295,o damn its trui agreeairplanes are awesome but...,o damn it trui agreeairplanes are awesome but ...
3282,misshoustonl,misshoustonl
105,just like that this is why ahahaha,just like that this is why ahahaha


In [51]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

We won't remove numerics because we could loose meaning of our text if we lost the numerics. We could also further prepare our text by performing spelling correction but this is a resource intensive process that we will skip for now.

#### Feature Engineering Techniques 

In [68]:
# Feature Construction: Length of tweet
# ---
# YOUR CODE GOES BELOW
#
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
df["length"]=df['text'].str.len()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [65]:
nltk.download('punkt')
# Feature Construction: Word count 
# ---
# YOUR CODE GOES BELOW
# 
df['words'] = df.text.apply(lambda x: " ".join(x for x in x.split())) 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
df.head()

,target,text,no_of_stopwords,lemmatization,length,words
0,0,switchfoot a thats a bummer you shoulda go...,8,switchfoot a thats a bummer you shoulda got da...,82,switchfoot a thats a bummer you shoulda got da...
1,0,obama forges his muslim alliance against the c...,9,obama forge his muslim alliance against the ci...,107,obama forges his muslim alliance against the c...
2,4,had the most spectacular prom ever but now my...,13,had the most spectacular prom ever but now my ...,130,had the most spectacular prom ever but now my ...
3,0,i am overwhelmed today taking a moment to eat...,5,i am overwhelmed today taking a moment to eat ...,55,i am overwhelmed today taking a moment to eat ...
4,0,lindork tres sad i was totally a max fan sy...,3,lindork tres sad i wa totally a max fan sytycd,50,lindork tres sad i was totally a max fan sytycd


In [72]:
pos_dic = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

def pos_check(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_dic[flag]:
                cnt += 1
    except:
        pass
    return cnt

In [78]:
def get_polarity(text):
    try:
        textblob = TextBlob(unicode(text, 'utf-8'))
        pol = textblob.sentiment.polarity
    except:
        pol = 0.0
    return pol

In [81]:
def get_subjectivity(text):
    try:
        textblob = TextBlob(unicode(text, 'utf-8'))
        subj = textblob.sentiment.subjectivity
    except:
        subj = 0.0
    return subj

In [89]:
# Feature Construction: Word density (Average no. of words / tweet)
df['word_count'] = df.text.apply(lambda x: len(str(x).split(" ")))


In [73]:
# Feature Construction: Noun count
df['nouns'] = df.text.apply(lambda x: pos_check(x, 'noun'))

In [74]:
# Feature Construction: Verb count
df['verbs'] = df.text.apply(lambda x: pos_check(x, 'verb'))

In [75]:
# Feature Construction: Adjective count / Tweet
df['adjs'] = df.text.apply(lambda x: pos_check(x, 'adj'))


In [76]:
# Feature Construction: Adverb count / Tweet
df['adverbs'] = df.text.apply(lambda x: pos_check(x, 'adv'))

In [77]:
# Feature Construction: Pronoun 
df['pronouns'] = df.text.apply(lambda x: pos_check(x, 'pron'))

In [82]:
# Feature Construction: Subjectivity
df['subjectivity'] = df.text.apply(get_subjectivity)


In [79]:
# Feature Construction: Polarity
df['polarity'] = df.text.apply(get_polarity)

In [84]:
# Feature Construction: Word Level N-Gram TF-IDF Feature 
from nltk import word_tokenize, ngrams
list(ngrams(word_tokenize(df['text'][0]), 2)) 

[('switchfoot', 'a'),
 ('a', 'thats'),
 ('thats', 'a'),
 ('a', 'bummer'),
 ('bummer', 'you'),
 ('you', 'shoulda'),
 ('shoulda', 'got'),
 ('got', 'david'),
 ('david', 'carr'),
 ('carr', 'of'),
 ('of', 'third'),
 ('third', 'day'),
 ('day', 'to'),
 ('to', 'do'),
 ('do', 'it'),
 ('it', 'd')]

In [86]:
# Feature Construction: Character Level N-Gram TF-IDF Feature
list(ngrams(df['text'][0], 2))


[(' ', 's'),
 ('s', 'w'),
 ('w', 'i'),
 ('i', 't'),
 ('t', 'c'),
 ('c', 'h'),
 ('h', 'f'),
 ('f', 'o'),
 ('o', 'o'),
 ('o', 't'),
 ('t', ' '),
 (' ', ' '),
 (' ', ' '),
 (' ', 'a'),
 ('a', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'a'),
 ('a', 't'),
 ('t', 's'),
 ('s', ' '),
 (' ', 'a'),
 ('a', ' '),
 (' ', 'b'),
 ('b', 'u'),
 ('u', 'm'),
 ('m', 'm'),
 ('m', 'e'),
 ('e', 'r'),
 ('r', ' '),
 (' ', ' '),
 (' ', 'y'),
 ('y', 'o'),
 ('o', 'u'),
 ('u', ' '),
 (' ', 's'),
 ('s', 'h'),
 ('h', 'o'),
 ('o', 'u'),
 ('u', 'l'),
 ('l', 'd'),
 ('d', 'a'),
 ('a', ' '),
 (' ', 'g'),
 ('g', 'o'),
 ('o', 't'),
 ('t', ' '),
 (' ', 'd'),
 ('d', 'a'),
 ('a', 'v'),
 ('v', 'i'),
 ('i', 'd'),
 ('d', ' '),
 (' ', 'c'),
 ('c', 'a'),
 ('a', 'r'),
 ('r', 'r'),
 ('r', ' '),
 (' ', 'o'),
 ('o', 'f'),
 ('f', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'i'),
 ('i', 'r'),
 ('r', 'd'),
 ('d', ' '),
 (' ', 'd'),
 ('d', 'a'),
 ('a', 'y'),
 ('y', ' '),
 (' ', 't'),
 ('t', 'o'),
 ('o', ' '),
 (' ', 'd'),
 ('d', 'o'),
 ('o', ' '),

In [90]:
df.shape

(10001, 14)

In [91]:
df.head(2)

,target,text,no_of_stopwords,lemmatization,length,words,nouns,verbs,adjs,adverbs,pronouns,polarity,subjectivity,word_count
0,0,switchfoot a thats a bummer you shoulda go...,8,switchfoot a thats a bummer you shoulda got da...,82,switchfoot a thats a bummer you shoulda got da...,4,5,2,0,2,0.0,0.0,21
1,0,obama forges his muslim alliance against the c...,9,obama forge his muslim alliance against the ci...,107,obama forges his muslim alliance against the c...,6,2,2,1,2,0.0,0.0,24


In [100]:
# Let's prepare the constructed features for modeling
# ---
#
X_metadata = np.array(df.iloc[:, 2:14])
X_metadata

array([[8,
        'switchfoot a thats a bummer you shoulda got david carr of third day to do it d',
        82, ..., 0.0, 0.0, 21],
       [9,
        'obama forge his muslim alliance against the civilized world and he didnt even drop in for a cup of tea',
        107, ..., 0.0, 0.0, 24],
       [13,
        'had the most spectacular prom ever but now my bed is serenading me and i must answer sweet dream my friend what a wonderful day',
        130, ..., 0.0, 0.0, 26],
       ...,
       [2, 'haha linas hyper already well lucky you im in college', 54,
        ..., 0.0, 0.0, 11],
       [1, 'omg really good day happened right here', 40, ..., 0.0, 0.0,
        8],
       [7,
        'love2cookpie i saw you on division and 68th but you didnt see me',
        66, ..., 0.0, 0.0, 15]], dtype=object)

In [109]:
# We combine our two tfidf (sparse) matrices and X_metadata
# ---

from sklearn.feature_extraction.text import TfidfVectorizer

tfidfw = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='word', ngram_range=(1,3),  stop_words= 'english')
df_word_vect = tfidfw.fit_transform(df['text'])
df_word_vect.toarray()

tfidfc = TfidfVectorizer(max_features=1000, lowercase=True, analyzer='char', ngram_range=(1,3),  stop_words= 'english')
df_char_vect = tfidfc.fit_transform(df['text'])
df_char_vect.toarray()

X = scipy.sparse.hstack([df_word_vect, df_char_vect
                        #  ,  X_metadata
                         ])
X

<10001x2000 sparse matrix of type '<class 'numpy.float64'>'
	with 1144375 stored elements in COOrdinate format>

In [106]:
df_word = pd.DataFrame(df_word_vect.toarray(), columns=tfidfw.get_feature_names_out())
df_char = pd.DataFrame(df_char_vect.toarray(), columns=tfidfc.get_feature_names_out())

In [107]:
df_char.head()

,,1,2,a,a,ab,ag,al,am,an,...,yin,yo,you,ys,ys,yy,z,â,ã,ãâ
0,0.375270,0.0,0.0,0.065793,0.114210,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.057267,0.059568,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.387250,0.0,0.0,0.115419,0.050089,0.0,0.089231,0.063912,0.000000,0.048077,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.066624,0.0,0.0,0.0
2,0.383070,0.0,0.0,0.071358,0.041290,0.0,0.000000,0.000000,0.000000,0.079264,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,0.310061,0.0,0.0,0.138620,0.080210,0.0,0.000000,0.000000,0.113599,0.076988,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.337476,0.0,0.0,0.050292,0.087301,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [110]:
# Getting our response variable
# ---
#
y = np.array(df.iloc[:, 0])
y

array([0, 0, 4, ..., 0, 4, 0])

### 4. Data Modelling

During this step, we will use machine learning algorithms to train and test our sentiment analysis models.

In [111]:
# Splitting our data
# ---
#
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [112]:
# Fitting our model
# ---
#

# Importing the algorithms
from sklearn.naive_bayes import MultinomialNB 
from sklearn.linear_model import LogisticRegression

nb_classifier = MultinomialNB() 
lr_classifier = LogisticRegression(max_iter=1000) 

# Training our model
nb_classifier.fit(X_train, y_train) 
lr_classifier.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [113]:
# Making predictions
# ---
#
y_predict_nb = nb_classifier.predict(X_test) 
y_predict_lr = lr_classifier.predict(X_test)

In [114]:
# Evaluating the Models
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Accuracy scores
# ---
#
print("Naive Bayes Classifier:\n", accuracy_score(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", accuracy_score(y_test, y_predict_lr))

Naive Bayes Classifier:
 0.7221389305347327
Logistic Regression Classifier: 
 0.7381309345327336


In [115]:
# Confusion matrices
# ---
# 
print("Naive Bayes Classifier: \n", confusion_matrix(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", confusion_matrix(y_test, y_predict_lr))

Naive Bayes Classifier: 
 [[734 242]
 [314 711]]
Logistic Regression Classifier: 
 [[731 245]
 [279 746]]


In [116]:
# Classification Reports
# ---
#
print("Naive Bayes Classifier: \n", classification_report(y_test, y_predict_nb)) 
print("Logistic Regression Classifier: \n", classification_report(y_test, y_predict_lr))

Naive Bayes Classifier: 
               precision    recall  f1-score   support

           0       0.70      0.75      0.73       976
           4       0.75      0.69      0.72      1025

    accuracy                           0.72      2001
   macro avg       0.72      0.72      0.72      2001
weighted avg       0.72      0.72      0.72      2001

Logistic Regression Classifier: 
               precision    recall  f1-score   support

           0       0.72      0.75      0.74       976
           4       0.75      0.73      0.74      1025

    accuracy                           0.74      2001
   macro avg       0.74      0.74      0.74      2001
weighted avg       0.74      0.74      0.74      2001



**Evaluation our Models**

* **Accuracy:** the percentage of texts that were assigned the correct topic.
* **Precision:** the percentage of texts the classifier classified correctly out of the total number of texts it predicted for each topic
* **Recall:** the percentage of texts the model predicted for each topic out of the total number of texts it should have predicted for that topic.
* **F1 Score:** the average of both precision and recall.

To improve our model, we can try perfoming other text processing techniques that would better prepare our data for fitting our model. We can also use different vectorizing techniques, implement other machine learning models and perform hyperparameter tuning.

### 5. Recommendations


Our best model had an accuracy of 73.25% and use it for classifying newer tweets. We can improve this performance by performing hyperparameter tuning and feature engineering methods. 